<a href="https://colab.research.google.com/github/Rohit-Singh1/NLP/blob/Text-Generation/Text_Generation_with_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Necessary Imports
import spacy
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.utils import to_categorical
from random import randint
from keras.preprocessing.sequence import pad_sequences
import random
import math

In [4]:
#Reading file given path
def readFile(path):
  with open(path) as fp:
    text = fp.read()
  return text

In [5]:
def removePunct(doc):
  return [token.text.lower() for token in nlp(doc) 
          if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n' ]

In [6]:
def createSeq(doc_tokens):
  text_seq=[]
  for i in range(seq_len,len(text_token)):
    text_seq.append(text_token[i-seq_len:i])
  
  return text_seq

In [43]:
#CREATING THE MODEL
def creatingModel(vocab_size,seq_len):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size+1,output_dim=int(np.log2(vocab_size)+1),input_length=seq_len))
  model.add(LSTM(7*seq_len,return_sequences=True))
  model.add(LSTM(7*seq_len))
  model.add(Dense(7*seq_len,activation='relu'))
  model.add(Dense(7*seq_len,activation='relu'))
  model.add(Dense(vocab_size+1,activation='softmax'))

  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
  model.summary()
  return model


In [46]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

In [8]:
doc = readFile('/content/moby_dick_four_chapters.txt')

In [9]:
nlp = spacy.load('en',disable=['parser','tagger','ner'])
nlp.max_length = 2*nlp.max_length

In [10]:
text_token = removePunct(doc)

In [11]:
len(text_token)

11338

In [12]:
#Generating sequence of 'w' words sentences
seq_len = 20 + 1
text_seq = []


In [13]:
text_seq = createSeq(text_token)

In [31]:
np.array(text_seq).shape

(11317, 21)

In [32]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_seq)
sequences = np.array(tokenizer.texts_to_sequences(text_seq))

In [33]:
vocab_size = len(tokenizer.word_index)

In [34]:
vocab_size

2717

In [35]:
#TRAINING AND RESULT LABEL SPLITTING
X = sequences[:,:-1]
y = sequences[:,-1]

In [36]:
y = to_categorical(y,num_classes=vocab_size+1)

In [37]:
seq_len = X.shape[1]

In [44]:
model = creatingModel(vocab_size,seq_len)


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 12)            32616     
_________________________________________________________________
lstm_8 (LSTM)                (None, 20, 140)           85680     
_________________________________________________________________
lstm_9 (LSTM)                (None, 140)               157360    
_________________________________________________________________
dense_12 (Dense)             (None, 140)               19740     
_________________________________________________________________
dense_13 (Dense)             (None, 140)               19740     
_________________________________________________________________
dense_14 (Dense)             (None, 2718)              383238    
Total params: 698,374
Trainable params: 698,374
Non-trainable params: 0
________________________________________________

In [45]:
# fit model
model.fit(X, y, batch_size=128, epochs=5,verbose=1)

Epoch 1/5
89/89 [==============================] - 18s 163ms/step - loss: 7.2354 - accuracy: 0.0463
Epoch 2/5
89/89 [==============================] - 14s 162ms/step - loss: 6.3516 - accuracy: 0.0502
Epoch 3/5
89/89 [==============================] - 14s 162ms/step - loss: 6.3227 - accuracy: 0.0513
Epoch 4/5
89/89 [==============================] - 14s 162ms/step - loss: 6.2227 - accuracy: 0.0521
Epoch 5/5
89/89 [==============================] - 14s 161ms/step - loss: 6.0951 - accuracy: 0.0583


In [51]:
random.seed(11)
random_pick = random.randint(0,len(text_seq))
random_seed_text = text_seq[random_pick]
seed_text = ' '.join(random_seed_text)
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=10)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'the the the the the the the the the the'

In [47]:
random.randint(0,12)

3